# 8. Considerando el pagelink_sample.csv, usando una representación de grafos obtener aquellos contenidos que tienen “relaciones no correspondidas”. Entendemos como funciona una relación correspondida con un ejemplo: Si el contenido A tiene un link al B, pero B no tiene un link a A, podemos decir que B tiene una relación no correspondida con A.  (***)

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
import pandas as pd
import numpy as np
import os

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [3]:
#df = pd.read_csv('Wikipedia dataset 2c2021/pagelinks_sample.csv', usecols = ["pl_from","pl_title"])

In [4]:
#df = df.sample(frac = 0.25)
#df.to_parquet('Wikipedia dataset 2c2021/pagelinks_sample.parquet')
#del df

### Tuve que filtrar el csv en pandas y tomar un samples tanto de plinks como que content porque si no no había forma de que lo levantara (con sample de 0.5 se colgaba)

In [5]:
pagelink_ori = sqlContext.read.parquet('Wikipedia dataset 2c2021/pagelinks_sample.parquet').rdd

In [6]:
pagelink_ori.take(2)

[Row(pl_from=6862922, pl_title='PatruBOT', __index_level_0__=22895178),
 Row(pl_from=7731553, pl_title='Instituto_de_Estudios_PolÃ\xadticos', __index_level_0__=29866549)]

In [7]:
content_ori = sqlContext.read.parquet('Wikipedia dataset 2c2021/contents.parquet').rdd
content_ori.take(2)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido')]

In [8]:
#content rdd: (title, id)
content = content_ori.map(lambda x: (x[0],x[1])).sample(False,0.6)
content.take(5)

[('Andorra', 7),
 ('Argentina', 10),
 ('Geografía de Andorra', 15),
 ('Demografía de Andorra', 17),
 ('Artes visuales', 19)]

In [9]:
#pagelink rdd, (title pointed, id that point)
pagelink = pagelink_ori.map(lambda x: (x[1],x[0]))
pagelink.take(2)

[('PatruBOT', 6862922), ('Instituto_de_Estudios_PolÃ\xadticos', 7731553)]

In [10]:
join = pagelink.join(content)
join.take(2)

[('Monje', (4693315, 7428024)), ('Monje', (715057, 7428024))]

In [11]:
#A ---> B , (A,B)
a_point_to_b = join.map(lambda x: (x[1][0],x[1][1]))
a_point_to_b.take(2)

[(4693315, 7428024), (715057, 7428024)]

In [12]:
#(A, (LIST B1, B2, B3, ETC))
a_point_to_list = a_point_to_b.groupByKey().map(lambda x: (x[0], list(x[1])))
a_point_to_list.take(2)

[(163056, [7428024, 20666, 6823, 4133027]), (4806120, [7428024, 63104])]

In [13]:
# (B <--- A)
b_point_by_a = a_point_to_b.map(lambda x: (x[1],x[0]))
b_point_by_a.take(2)

[(7428024, 4693315), (7428024, 715057)]

In [14]:
#B, A, (B point to list)
final_join = b_point_by_a.join(a_point_to_list)
final_join.take(2)

[(7428024,
  (4693315,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (715057,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797]))]

In [15]:
#se fija si x se encuentra en list. Si se encuentra devuelve false, de lo contrario true.
def funcion(x,list):
    for i in list:
        if (i == x):
            return False
    return True

In [16]:
resultado = final_join.filter(lambda x: funcion(x[1][0],x[1][1]))

In [17]:
resultado.take(50)

[(7428024,
  (4693315,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (715057,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (9640950,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (241325,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (5422609,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (1006445,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (4734094,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (163056,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (5223668,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 5291797])),
 (7428024,
  (4806120,
   [111033, 48082, 1173, 1834856, 723953, 692766, 617378, 611531, 52917